<a href="https://colab.research.google.com/github/roman6s/SCM_Fallstudie/blob/main/Fallstudie_Version_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie BESS

In [29]:
! git clone https://github.com/AlexKressner/WS24_Supply_Chain_Optimierung

fatal: destination path 'WS24_Supply_Chain_Optimierung' already exists and is not an empty directory.


In [30]:
! pip install -q pyscipopt

In [31]:
import pandas as pd
from pyscipopt import Model, quicksum

## Optimierungsmodell

In [32]:
scip = Model()

## Daten laden

In [33]:
folder = "WS24_Supply_Chain_Optimierung/Daten/Fallstudie"

In [34]:
# Preisprogonose
preisprognose = pd.read_excel(f"{folder}/Preisprognosen.xlsx")


##Indexmenge

$T = \{1, 2, \ldots, 24\} \quad \text{(Menge der Stunden des Folgetages)}$

In [35]:
T = range (1,25)

##Parameter

In [36]:
cap       = 40.0            # Nominelle Kapazität in [MWh]

In [37]:
DoD       = 0.80            # Depth of Discharge

In [38]:
SOC_min   = cap*(1 - DoD)   # Minimaler Ladezustand in [MWh]

In [39]:
SOC_max   = cap             # Maximaler Ladezustand in [MWh]

In [40]:
c_rate    = 0.5             # C-Rate

In [41]:
eta_rte = 0.975             # Round-trip Efficiency (einfache Richtung)

In [42]:
eta_wr    = 0.985           # Wirkungsgrad Wechselrichter

In [43]:
cyclecost = 1500.0          # Fixkosten pro Zyklusdurchlauf

In [44]:
maxCycles = 2.0             # Maximal durchführbare Zyklen

In [45]:
p_Markt = preisprognose.groupby("Stunde")["Strompreis"].mean().tolist()    # Marktpreiserwartung zur Stunde t

In [46]:
# Lade- und Entladeverluste trennen
# RTE für jede Richtung:
eta_charge    = eta_rte * eta_wr  # Lade-Verlust
eta_discharge = eta_rte * eta_wr  # Entlade-Verlust

In [47]:
# Zählvariable für verbrauchte Zyklen
Z = scip.addVar(
      vtype="CONTINUOUS",
      lb=0,
      ub=maxCycles,
      name="cycles_used"
    )

##Entscheidungsvariablen

$\text{SOC}_t \in [8, 40] \quad [\text{MWh}], \text{ Ladezustand der Batterie am Ende von Stunde } t$


In [48]:
# State of Charge je Stunde
SOC = {t: scip.addVar(
             vtype="CONTINUOUS",
             lb=SOC_min,
             ub=SOC_max,
             name=f"SOC_{t}"
          )
       for t in T}

$0 \leq \text{charge}_t \leq 20 \quad [\text{MWh}], \text{ Ladeleistung in Stunde } t.$


In [49]:
# Ladestrom je Stunde (jeweils in MWh, nicht negativ)
charge = {t: scip.addVar(
               vtype="CONTINUOUS",
               lb=0,
               ub=c_rate * cap,
               name=f"charge_{t}"
            )
          for t in T}

$0 \leq \text{discharge}_t \leq 20 \quad [\text{MWh}], \text{ Entladeleistung in Stunde  } t.$


In [50]:
# Entladestrom je Stunde (jeweils in MWh, nicht negativ)
discharge = {t: scip.addVar(
                  vtype="CONTINUOUS",
                  lb=0,
                  ub=c_rate * cap,
                  name=f"discharge_{t}"
               )
             for t in T}

##Nebenbedingungen

$\text{SOC}_1 = 0.5 \cdot \text{cap}, \quad \text{SOC}_{24} = 0.5 \cdot \text{cap}$


In [51]:
# (1) Anfangs- und End-SOC = 50% Kapazität
scip.addCons(SOC[1] == 0.5 * cap)
scip.addCons(SOC[24] == 0.5 * cap)

# Verhindert Entladung bei t=24
scip.addCons(discharge[24] == 0)

c3

$\text{SOC}_t = \text{SOC}_{t-1} + \eta_{\text{charge}} \cdot \text{charge}_{t-1} - \frac{1}{\eta_{\text{discharge}}} \cdot \text{discharge}_{t-1}$


In [52]:
# (2) Ladezustandsdynamik
for t in T:
    if t == 1:
        continue
    scip.addCons(
        SOC[t]
        == SOC[t-1]
           + eta_charge     * charge[t-1]
           - (1/eta_discharge)* discharge[t-1]
    )

$
\frac{\sum_{t \in T} (\text{charge}_t + \text{discharge}_t)}{2 \cdot \text{cap} \cdot \text{DoD}} \leq Z \leq \text{maxCycles}
$





In [53]:
# (3) Zyklenbegrenzung
#     Ein Vollzyklus = 2*(cap * DoD). Also berechnen wir den gesamten "Throughput"
#     und setzen Z >= Throughput / [2 * (cap*DoD)].
throughput = quicksum(charge[t] + discharge[t] for t in T)
scip.addCons(
    Z >= throughput / (2.0 * cap * DoD)
)
# Z soll <= 2 sein
scip.addCons(
    Z <= maxCycles
)

c28

##Zielfunktion

$Max \quad \underbrace{\sum_{t \in T} \left[ p_t \cdot (\text{discharge}_t) - p_t \cdot (\text{charge}_t) \right]}_{\text{Erlöse - Kosten für Strom}}
-
\underbrace{\text{cost} \cdot Z}_{\text{Zykluskosten}}
$


In [54]:
#   Erlöse - Kosten pro Zyklus
scip.setObjective(quicksum(
    p_Markt[t-1] * discharge[t] - p_Markt[t-1] * charge[t]
    for t in T
) - cyclecost * Z, sense="maximize")

##Berechnung des Ergebnis

In [55]:
# Modell lösen
scip.optimize()

In [56]:
# Ausgabe der Ergebnisse
print(f"Optimaler Zielfunktionswert: {scip.getObjVal():.2f} €")

print("\nStundenweise Lösung:")
for t in T:
    soc_val     = scip.getVal(SOC[t])
    chg_val     = scip.getVal(charge[t])
    dis_val     = scip.getVal(discharge[t])
    print(f"  t={t:2d}: "
          f"SOC={soc_val:5.2f}  "
          f"charge={chg_val:5.2f}  "
          f"discharge={dis_val:5.2f}  "
          f"Preis={p_Markt[t-1]:6.2f}")

print(f"\nEffektiv genutzte Zyklen: {scip.getVal(Z):.2f}")

Optimaler Zielfunktionswert: 1885.12 €

Stundenweise Lösung:
  t= 1: SOC=20.00  charge= 0.00  discharge= 0.00  Preis= 81.52
  t= 2: SOC=20.00  charge= 0.00  discharge= 0.00  Preis= 72.24
  t= 3: SOC=20.00  charge= 0.00  discharge= 0.00  Preis= 68.16
  t= 4: SOC=20.00  charge= 0.00  discharge= 0.00  Preis= 66.64
  t= 5: SOC=20.00  charge= 0.00  discharge= 0.00  Preis= 66.80
  t= 6: SOC=20.00  charge= 0.00  discharge= 0.00  Preis= 71.04
  t= 7: SOC=20.00  charge= 0.00  discharge= 0.00  Preis= 88.56
  t= 8: SOC=20.00  charge= 0.00  discharge=11.52  Preis= 92.68
  t= 9: SOC= 8.00  charge= 0.00  discharge= 0.00  Preis= 80.92
  t=10: SOC= 8.00  charge= 0.00  discharge= 0.00  Preis= 61.08
  t=11: SOC= 8.00  charge= 0.00  discharge= 0.00  Preis= 43.08
  t=12: SOC= 8.00  charge= 0.00  discharge= 0.00  Preis= 30.88
  t=13: SOC= 8.00  charge= 0.00  discharge= 0.00  Preis= 22.44
  t=14: SOC= 8.00  charge=13.32  discharge= 0.00  Preis= 15.96
  t=15: SOC=20.79  charge=20.00  discharge= 0.00  Preis= 